In [39]:
## Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as shpgeo
import shapely.ops as shpops
from random import sample
import requests
import polyline

In [2]:
# import os
# os.getcwd()

In [3]:
## EV Project Process Script ----
## Read in all_roads shape file
## EPSG 4326 corresponds to geospatial data across a standardized coordinate system (WGS84) for Earth
## EPSG 2231 is a subset of 4326 specific to regions covering Colorado using UTM vs. Lat/Long
## Re-evaluating EPSG 2231 because it doesn't cover all of Colorado
## EPSG 2232 corresponds to central Colorado - RIP
## EPSG 32613 corresponds to UTM 13N which covers all of Colorado
# all_roads = gpd.read_file('/home/jcarey9/ev_chargers/data/all_roads.shp').to_crs("GCS_WGS84")
all_roads = gpd.read_file('C:/Users/jason/Downloads/Datasets/ev_geospatial/all_roads.shp').to_crs('GCS_WGS84')
all_roads_proj = all_roads.to_crs("EPSG:32613")

## Read in chgs shape file
# chgs = gpd.read_file('/home/jcarey9/ev_chargers/data/alt_fuel_stations.shp').to_crs("GCS_WGS84")
chgs = gpd.read_file('C:/Users/jason/Downloads/Datasets/ev_geospatial/alt_fuel_stations.shp').to_crs('GCS_WGS84')
chgs_proj = chgs.to_crs("EPSG:32613")

## Create initial constants
route_dist = 402336
fuel_dist = route_dist * .25
n_sim = 3

## Get the CRS for future calculations between geometries
wgs = all_roads.crs
epsg = all_roads_proj.crs

## Run simulation
# sim_outcomes = route_process(all_roads, chgs, route_dist, fuel_dist, n_sim)

## Save outcomes to file
# pd.to_csv('/home/jcarey9/ev_chargers/data/ev_sim_outcomes.csv')

In [40]:
## Create sample point function for starting and ending positions ----
# from random import sample
# import geopandas as gpd
# import shapely.geometry as shp

def sample_point(data, crs):
    
    data = all_roads_proj.copy()
    
    ## sample one random row from 'data'
    sample_road_row = sample(range(len(data)), 1)

    ## Filter down 'data' to sample road
    sample_road_data = data.loc[sample_road_row].reset_index(drop = True)

    ## Randomly select a pair of coordinates within the road we've selected as a starting point
    sample_road_coords = list(sample_road_data.geometry[0].coords)
    sample_index = sample(range(len(sample_road_coords)), 1)
    sample_geo = shpgeo.Point(list(sample_road_coords[sample_index[0]]))

    point_df = gpd.GeoDataFrame({'geometry': [sample_geo]}, crs = "EPSG:32613")
    point_df = point_df.to_crs(crs)

    ## Return a list with the corresponding sample road row number in 'data' and a geometry object
    return({
        "road_row_num" : sample_road_data.record[0], 
        "point" : point_df
    })

In [41]:
## Get starting point function ----
def get_start_point(df, crs):
    
    data = df.copy()

    # filter only roadways of level 3
    valid_startpoints = data[data["level"] == 3].reset_index(drop = True)

    return sample_point(valid_startpoints, crs)

In [42]:
## Get ending point function ----
def get_end_point(df, start_pos, route_dist, crs):
    
    ## Testing/CRS translation if given wrong data set
    # data = all_roads_proj
    # start_pos = get_start_point(all_roads, epsg)

    # start_pos['point']

    # if (data.crs.utm_zone == None):
    #     data = data.to_crs("EPSG:32613")
    
    data = df.copy()
    
    sp = start_pos['point'].to_crs("EPSG:32613")
    
    # calculate distances from start_position, these are in meters since we are using EPSG:32613
    data['distances'] = data['geometry'].distance(sp.geometry[0])
    # min(data['distances'])
    
    # filter only distances that are further than half route_dist
    valid_endpoints = data[data['distances'] >= (route_dist / 2)].reset_index(drop = True)
    
    return sample_point(valid_endpoints, crs)

In [103]:
def calculate_route(pickup, dropoff, crs, return_trip = False):
    ## Convert pickup coordinates from UTM to CRS specified
    pickup_lon = pickup['point'].to_crs(crs).iloc[0]['geometry'].x
    pickup_lat = pickup['point'].to_crs(crs).iloc[0]['geometry'].y
    
    ## Convert dropoff coordinates from UTM to CRS specified
    dropoff_lon = dropoff['point'].to_crs(crs).iloc[0]['geometry'].x
    dropoff_lat = dropoff['point'].to_crs(crs).iloc[0]['geometry'].y
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc + "?overview=full&annotations=true")
    if r.status_code!= 200:
        return {}
  
    res = r.json()
    
    ## Get route details from response
    route_distances = [0, *res['routes'][0]['legs'][0]['annotation']['distance']]
    
    ## Collect main features of route from response
    route_coords = polyline.decode(res['routes'][0]['geometry'])
    route = gpd.GeoSeries([shpgeo.Point((x[1], x[0])) for x in route_coords], crs = wgs)
    
    ## Create route details data frame with distances and coords
    route_detail = pd.DataFrame({"distance": route_distances, "route": route})
    
    ## Return trip decision
    if return_trip == True:
        ## Generate a round trip by selecting all but last row and appending to route df
        ## Get return points and reverse order
        rd_return = route_detail.loc[0:len(route_detail) - 2][::-1]
        ## Row bind to route detail df
        route_detail_rb = pd.concat([route_detail, rd_return]).reset_index(drop = True)
        route_detail_rb['total_distance'] = route_detail_rb['distance'].cumsum()
        route_df = gpd.GeoDataFrame({
            'dist': route_detail_rb['distance'],
            'total_dist': route_detail_rb['total_distance'],
            'geometry': route_detail_rb['route']
        }, crs = wgs)
    else:
        route_detail['total_distance'] = route_detail['distance'].cumsum()
        route_df = gpd.GeoDataFrame({
            'dist': route_detail['distance'],
            'total_dist': route_detail['total_distance'],
            'geometry': route_detail['route']
        }, crs = wgs)
    
    start_point = gpd.GeoSeries(shpgeo.Point(res['waypoints'][0]['location'][0], res['waypoints'][0]['location'][1]), crs = wgs)
    end_point = gpd.GeoSeries(shpgeo.Point(res['waypoints'][1]['location'][0], res['waypoints'][1]['location'][1]), crs = wgs)
    distance = res['routes'][0]['distance']
    
    out = {'route':route,
           'route_detail':route_df,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [106]:
## Route process function ----
# def route_process(road_data, evc_data, route_dist, fuel_dist, n_sim = 1, crs):
    
# for i in list(range(1, n_sim + 1)):
#     print(i)

start_pos = get_start_point(all_roads_proj, wgs)
end_pos = get_end_point(all_roads_proj, start_pos, route_dist, wgs)

print(start_pos['point'])
print(end_pos['point'])

route = calculate_route(start_pos, end_pos, wgs, return_trip = True)

print(route['route'])
print(route['route_detail'])

## If route['distance'] < route_dist then skip simulation otherwise
# simulate_trip():

                      geometry
0  POINT (-105.39289 37.41835)
                      geometry
0  POINT (-107.54981 39.55792)
0       POINT (-105.39291 37.41834)
1       POINT (-105.39341 37.41912)
2       POINT (-105.39356 37.41947)
3       POINT (-105.39241 37.42152)
4       POINT (-105.39205 37.42243)
                   ...             
5536    POINT (-107.54742 39.55982)
5537    POINT (-107.54832 39.55892)
5538    POINT (-107.54925 39.55820)
5539    POINT (-107.54970 39.55801)
5540    POINT (-107.54972 39.55791)
Length: 5541, dtype: geometry
             dist     total_dist                     geometry
0        0.000000       0.000000  POINT (-105.39291 37.41834)
1       96.956596      96.956596  POINT (-105.39341 37.41912)
2       40.904443     137.861039  POINT (-105.39356 37.41947)
3      249.105586     386.966625  POINT (-105.39241 37.42152)
4      106.652115     493.618740  POINT (-105.39205 37.42243)
...           ...            ...                          ...
11076  106.65211

In [164]:
## Simulate trip function ----
# def simulate_trip(route, start_pos, end_pos, evc_data, route_dist, fuel_dist, alpha = 2):

alpha = 2

sim_route = route['route']
sim_route_details = route['route_detail']

rng = route_dist
route_index = 730
trip_direction = 1

# while rng > 0 & trip_direction < 3:
# while rng > 0:
    
#     ## Decrease range
#     rng -= sim_route_details['dist'].loc[route_index]
    
#     ## Check if we need to refuel
#     if rng < fuel_dist:
        
#         ## Get our current location and find the nearest charger
#         current_location = sim_route_details['geometry'].loc[route_index]
#         nearest_charger = nearest_chg_pt(current_location, evc_data)
        
#         ## Find the number of chargers at the charging station
#         num_chgs = int(evc_data[evc_data['geometry'] == near_chg]['nm_chrg'])
        
#         ## Use a Poisson RV to estimate number of available chargers at the station
#         in_use = np.random.poisson(alpha, 1)
        
#         if in_use >= num_chgs:
#             charger_available = False
#         else:
#             charger_available = True
        
#         if charger_available == True:
#             ## Get coordinates for current location and charging station
#             pos1 = format_coord(current_location, wgs)
#             pos2 = format_coord(nearest_charger, wgs)

#             ## Generate new route to charger
#             re_route = calculate_route(pos1, pos2, wgs, return_trip = False)

#             ## Simulate travel to the charger
#             for i in range(len(re_route['route_detail'])):
#                 rng -= re_route['route_detail'].loc[i]['dist']
#                 if rng <= 0:
#                     outcome = pd.DataFrame({
#                         "fll_ddr": None,
#                         "nm_chrg": 4,
#                         "fal_typ": "Chargers out of range",
#                         "geometry": re_route['route_detail'].loc[i]
#                     })
#                     return outcome
            
#             ## Refuel the vehicle
#             rng = route_dist
            
#             ## Re-route to the next point of interest (starting or ending position)
            
#         else:
#             ## Charger is unavailable
#             if rng <= 0:
#                 outcome = pd.DataFrame({
#                     "fll_ddr": None,
#                     "nm_chrg": 4,
#                     "fal_typ": "Chargers unavailable",
#                     "geometry": gpd.GeoSeries(current_location, crs = wgs)
#                 })
#                 return outcome
        
        
#     route_index += 1



In [145]:
def nearest_chg_pt(pos1, evc_data):
    chg_pts = evc_data['geometry'].unary_union
    return gpd.GeoSeries(shpops.nearest_points(pos1, chg_pts))[1]

def format_coord(pos, crs):
    return {'point': gpd.GeoDataFrame({'geometry': [pos]}, crs = crs)}

In [171]:
test = sim_route_details['geometry'].loc[route_index]
## starting point/current location
# print(test)

# evc_data = chgs
# print(evc_data['geometry'])
chg_pts = evc_data.geometry.unary_union

## This is my nearest charger
# print(gpd.GeoSeries(shpops.nearest_points(test, chg_pts))[1])
near_chg = nearest_chg_pt(test, evc_data)
# print(near_chg)

## Number of chargers at the nearest location
num_chgs = int(evc_data[evc_data['geometry'] == near_chg]['nm_chrg'])
print("Number of Chgs:", num_chgs)

in_use = np.random.poisson(alpha, 1)
print("In use:", in_use)

if in_use >= num_chgs:
    charger_available = False
else:
    charger_available = True

# print(gpd.GeoSeries(test, crs = wgs))
# print(gpd.GeoSeries(near_chg, crs = wgs))
print("Available:", charger_available)
  
# print(type(start_pos['point']))
# print(start_pos['point'])
# print(type(pos1['point']))
if charger_available == True:
    pos1 = format_coord(test, wgs)
#     print(pos1)
    pos2 = format_coord(near_chg, wgs)
#     print(pos2)
    re_route = calculate_route(pos1, pos2, wgs, return_trip = False)

    for i in range(len(re_route['route_detail'])):
        rng -= re_route['route_detail'].loc[i]['dist']
        if rng <= 0:
            outcome = pd.DataFrame({
                "fll_ddr": None,
                "nm_chrg": 4,
                "fal_typ": "Chargers out of range",
                "geometry": re_route['route_detail'].loc[i]
            })
#             return outcome

if route_index < len(sim_route) & trip_direction == 1:
    
    ## Generate new route from current location to next point of interest
    sim_route = calculate_route(
        format_coord(near_chg, wgs),
        end_pos,
        wgs,
        return_trip = False
    )
    
    ## Reset the route index
    route_index = 0

print(route_index)
print(len(sim_route))
print(trip_direction)

#     print(re_route['route_detail'].loc[4]['dist'])
    
#     print(len(re_route['route_detail']))
#     print(re_route)

Number of Chgs: 1
In use: [2]
Available: False
730
5541
1


0       POINT (-105.39291 37.41834)
1       POINT (-105.39341 37.41912)
2       POINT (-105.39356 37.41947)
3       POINT (-105.39241 37.42152)
4       POINT (-105.39205 37.42243)
                   ...             
5536    POINT (-107.54742 39.55982)
5537    POINT (-107.54832 39.55892)
5538    POINT (-107.54925 39.55820)
5539    POINT (-107.54970 39.55801)
5540    POINT (-107.54972 39.55791)
Length: 5541, dtype: geometry

In [10]:
## Convert pickup coordinates from UTM to CRS specified
pickup = start_pos
dropoff = end_pos
crs = wgs
return_trip = True

pickup_lon = pickup['point'].to_crs(crs).iloc[0]['geometry'].x
pickup_lat = pickup['point'].to_crs(crs).iloc[0]['geometry'].y

## Convert dropoff coordinates from UTM to CRS specified
dropoff_lon = dropoff['point'].to_crs(crs).iloc[0]['geometry'].x
dropoff_lat = dropoff['point'].to_crs(crs).iloc[0]['geometry'].y

loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
## API Documentation: http://project-osrm.org/docs/v5.24.0/api/#route-service
url = "http://router.project-osrm.org/route/v1/driving/"
r = requests.get(url + loc + "?overview=full&annotations=true")

# print(r)

res = r.json()
# print(res)

# print(res['routes'][0]['legs'][0]['annotation'])
# keys = ['nodes', 'distance']
# route_detail_df = {k: route_detail[k] for k in keys}
# route_detail_len = {k: len(route_detail[k]) for k in keys}

route_distances = [0, *res['routes'][0]['legs'][0]['annotation']['distance']]
# print(route_detail)

## TODO (5/11/22) - the response should have additional fields with data we want to pull, such as distance between points
route_init = polyline.decode(res['routes'][0]['geometry'])
# print(routes)
route = gpd.GeoSeries([shp.Point((x[1], x[0])) for x in route_init], crs = wgs)

route_detail = pd.DataFrame({"distance": route_distances, "route": route})

sum(route_detail['distance']) < route_dist

# print(len(route_detail))
# print(route_detail)
# rd_return = route_detail.loc[0:len(route_detail) - 2][::-1]
# rd = pd.concat([route_detail, rd_return]).reset_index(drop = True)
# print(rd)
# route_detail[::-1]

# if return_trip == True:
#     ## Get return points and reverse order
#     rd_return = route_detail.loc[0:len(route_detail) - 2][::-1]
#     ## Row bind to route detail df
#     route_detail_rb = pd.concat([route_detail, rd_return]).reset_index(drop = True)
#     route_detail_rb['total_distance'] = route_detail_rb['distance'].cumsum()
#     rd = gpd.GeoDataFrame({
#         'dist': route_detail_rb['distance'],
#         'total_dist': route_detail_rb['total_distance'],
#         'geometry': route_detail_rb['route']
#     }, crs = wgs)
# else:
#     route_detail['total_distance'] = route_detail['distance'].cumsum()
#     rd = gpd.GeoDataFrame({
#         'dist': route_detail['distance'],
#         'total_dist': route_detail['total_distance'],
#         'geometry': route_detail['route']
#     }, crs = wgs)

# start_point = gpd.GeoSeries(shp.Point(res['waypoints'][0]['location'][0], res['waypoints'][0]['location'][1]), crs = wgs)
# print(start_point)
# end_point = gpd.GeoSeries(shp.Point(res['waypoints'][1]['location'][0], res['waypoints'][1]['location'][1]), crs = wgs)
# print(end_point)
# distance = res['routes'][0]['distance']
# print(distance)

# print(rd)
# rd_epsg = rd.to_crs(epsg)
# print(rd_epsg)

# out = {
#     'route':route,
#     'route_detail':rd,
#     'start_point':start_point,
#     'end_point':end_point,
#     'distance':distance
# }

True

In [ ]:
# print(route['distance'])
route

# route_details_init = gpd.GeoDataFrame({'geometry': [shp.Point(x) for x in route['route']]}, crs = wgs)

# ## Trying to split each data frame at first
# # rd_offset = gpd.GeoDataFrame({'geometry': route_details_init['geometry'].shift(-1)}, crs = wgs)
# # rd_offset['geometry'] = rd_offset.where(rd_offset['geometry'] == None, route_details_init['geometry'], axis = 0)
# # print(rd_offset)

# ## Combined them
# route_details_init['lead'] = route_details_init['geometry'].shift(-1)
# route_details_init['lead'] = route_details_init.where(route_details_init['lead'] == None, route_details_init['geometry'], axis = 0)
# route_details_init['lead'] = gpd.GeoSeries(route_details_init['lead'], crs = wgs)

# print(route_details_init)

# ## To calculate the distances between each of the series I've created (geometry and lead), we would need to switch to a projected CRS
# ## Doing so converts all points in geometry to (Inf, Inf) which is impossible calculate on?
# route_details_init['geometry']

# route_details_init['distances'] = route_details_init['geometry'].distance(route_details_init['lead'], align = False)





In [ ]:
test = gpd.GeoSeries([shp.Point(-105.211, 38.962)], crs = wgs)
## TODO (5/11/22) - Points must be passed following (long, lat) not (lat, long) otherwise will return (inf, inf)
test.to_crs(epsg = 32613)
